In [2]:
from meta_kg.utils.py_io import read_jsonl, read_json

In [3]:
train_data = read_jsonl("data/proofwriter_owa_natlang/dev.jsonl")

In [5]:
train_data[0]['triples']

{'triple1': {'text': 'Bob is green.',
  'representation': '("Bob" "is" "green" "+")'},
 'triple2': {'text': 'Bob is blue.',
  'representation': '("Bob" "is" "blue" "+")'},
 'triple3': {'text': 'Eric is round.',
  'representation': '("Eric" "is" "round" "+")'},
 'triple4': {'text': 'Eric is rough.',
  'representation': '("Eric" "is" "rough" "+")'},
 'triple5': {'text': 'Eric is green.',
  'representation': '("Eric" "is" "green" "+")'},
 'triple6': {'text': 'Gary is rough.',
  'representation': '("Gary" "is" "rough" "+")'},
 'triple7': {'text': 'Gary is red.',
  'representation': '("Gary" "is" "red" "+")'},
 'triple8': {'text': 'Gary is nice.',
  'representation': '("Gary" "is" "nice" "+")'},
 'triple9': {'text': 'Gary is blue.',
  'representation': '("Gary" "is" "blue" "+")'}}

In [6]:
train_data[0]['rules']

{'rule1': {'text': 'If someone is round and red then they are young.',
  'representation': '((("someone" "is" "round" "+") ("someone" "is" "red" "+")) -> ("someone" "is" "young" "+"))'},
 'rule2': {'text': 'If someone is green and cold and red then they are blue.',
  'representation': '((("someone" "is" "green" "+") ("someone" "is" "cold" "+") ("someone" "is" "red" "+")) -> ("someone" "is" "blue" "+"))'},
 'rule3': {'text': 'If someone is nice and blue then they are round.',
  'representation': '((("someone" "is" "nice" "+") ("someone" "is" "blue" "+")) -> ("someone" "is" "round" "+"))'},
 'rule4': {'text': 'If someone is red and young and blue then they are cold.',
  'representation': '((("someone" "is" "red" "+") ("someone" "is" "young" "+") ("someone" "is" "blue" "+")) -> ("someone" "is" "cold" "+"))'},
 'rule5': {'text': 'If someone is big and round and young then they are nice.',
  'representation': '((("someone" "is" "big" "+") ("someone" "is" "round" "+") ("someone" "is" "young"

In [10]:
for question in train_data[0]['questions'].values():
    if question["answer"] == "Unknown":
        print(question["question"])

Bob is not round.
Dave is blue.
Bob is not big.
Dave is nice.
Bob is not red.
Dave is rough.
Dave is not red.
Eric is kind.


In [16]:
label_counter = {
    "True": 0,
    "False": 0,
    "Unknown": 0
}
for data in train_data:
    questions = list(data["questions"].values())
    for question in questions:
        label = question["answer"]
        label_counter[str(label)] += 1
label_counter


{'True': 7003, 'False': 7003, 'Unknown': 14004}

In [7]:
from meta_kg.dataset import MetaQADataReader
from argparse import Namespace

import itertools

config = Namespace(**{
    "data_dir": "data/proofwriter_owa_natlang",
    "input_format": "lm",
})

reader = MetaQADataReader()
train_data = reader.jsonl_file_reader("data/proofwriter_owa_natlang/train.jsonl", config)


print_data = {}
print_out_keys = set(
    itertools.chain(*[list(i.keys()) for i in train_data])
)

for key_name in print_out_keys:
    raw_data = [t[key_name]
                if key_name in t else [] for t in train_data]
    print_data[key_name] = [t for t in itertools.chain(*raw_data)]

i = 0
for data in train_data:
    if data["qa_pairs"] != None:
        i += len(data["qa_pairs"])
print(i)
len(print_data["qa_pairs"])

21008


21008

In [24]:
from pprint import pprint
from collections import Counter
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

eval_out = read_json("./output/dev_eval_out-epoch=0_step=28010_20221031-021656.json")

acc_class = {
    "true": 0,
    "false": 0,
    "unknown": 0,
}

targets = [data["answer"] for data in eval_out]
outputs = [data["gen_out"].strip() for data in eval_out]

class_count = Counter(targets)
pprint(class_count)

if targets and outputs and len(targets) == len(outputs):
    for label, gen in zip(targets, outputs):
        if gen.strip() == label:
            acc_class[label] += 1
        
    for key in acc_class:
        if class_count[key] > 0:
            acc_class[key] /= class_count[key]
        else:
            acc_class[key] = 0

    acc = accuracy_score(targets, outputs)
    f1 = f1_score(targets, outputs, average="macro")

pprint({
    "acc": acc,
    "f1": f1,
    "acc_true": acc_class["true"],
    "acc_false": acc_class["false"],
    "acc_unknown": acc_class["unknown"],
})

Counter({'true': 2002, 'false': 2002})
{'acc': 0.495004995004995,
 'acc_false': 0.0924075924075924,
 'acc_true': 0.8976023976023976,
 'acc_unknown': 0,
 'f1': 0.26491783733450075}


In [6]:
batch = [
    {'guid': 'd3747655-a840-434c-a1e2-af840c2d7636', 'qa_pairs': [('Is it true or false that Bob is blue?', 'true')], 'facts': [('To determine if Bob is blue, a person needs to know', 'Bob is green.'), ('To determine if Bob is blue, a person needs to know', 'Bob is blue.'), ('To determine if Bob is blue, a person needs to know', 'Eric is round.'), ('To determine if Bob is blue, a person needs to know', 'Eric is rough.'), ('To determine if Bob is blue, a person needs to know', 'Eric is green.'), ('To determine if Bob is blue, a person needs to know', 'Gary is rough.'), ('To determine if Bob is blue, a person needs to know', 'Gary is red.'), ('To determine if Bob is blue, a person needs to know', 'Gary is nice.'), ('To determine if Bob is blue, a person needs to know', 'Gary is blue.'), ('To determine if Bob is blue, a person needs to know', 'If someone is round and red then they are young.'), ('To determine if Bob is blue, a person needs to know', 'If someone is green and cold and red then they are blue.'), ('To determine if Bob is blue, a person needs to know', 'If someone is nice and blue then they are round.'), ('To determine if Bob is blue, a person needs to know', 'If someone is red and young and blue then they are cold.'), ('To determine if Bob is blue, a person needs to know', 'If someone is big and round and young then they are nice.'), ('To determine if Bob is blue, a person needs to know', 'If someone is green and blue and red then they are big.')]}, {'guid': 'd3747655-a840-434c-a1e2-af840c2d7636', 'qa_pairs': [('Is it true or false that Eric is not round?', 'false')], 'facts': [('To determine if Eric is not round, a person needs to know', 'Bob is green.'), ('To determine if Eric is not round, a person needs to know', 'Bob is blue.'), ('To determine if Eric is not round, a person needs to know', 'Eric is round.'), ('To determine if Eric is not round, a person needs to know', 'Eric is rough.'), ('To determine if Eric is not round, a person needs to know', 'Eric is green.'), ('To determine if Eric is not round, a person needs to know', 'Gary is rough.'), ('To determine if Eric is not round, a person needs to know', 'Gary is red.'), ('To determine if Eric is not round, a person needs to know', 'Gary is nice.'), ('To determine if Eric is not round, a person needs to know', 'Gary is blue.'), ('To determine if Eric is not round, a person needs to know', 'If someone is round and red then they are young.'), ('To determine if Eric is not round, a person needs to know', 'If someone is green and cold and red then they are blue.'), ('To determine if Eric is not round, a person needs to know', 'If someone is nice and blue then they are round.'), ('To determine if Eric is not round, a person needs to know', 'If someone is red and young and blue then they are cold.'), ('To determine if Eric is not round, a person needs to know', 'If someone is big and round and young then they are nice.'), ('To determine if Eric is not round, a person needs to know', 'If someone is green and blue and red then they are big.')]},
    {'guid': '8c6a277d-a1bf-46da-a984-89f8c2193b10', 'qa_pairs': [('Is it true or false that Charlie is kind?', 'true')], 'facts': [('To determine if Charlie is kind, a person needs to know', 'Charlie is kind.'), ('To determine if Charlie is kind, a person needs to know', 'Charlie is green.'), ('To determine if Charlie is kind, a person needs to know', 'Charlie is cold.'), ('To determine if Charlie is kind, a person needs to know', 'Charlie is blue.'), ('To determine if Charlie is kind, a person needs to know', 'Fred is red.'), ('To determine if Charlie is kind, a person needs to know', 'Fred is kind.'), ('To determine if Charlie is kind, a person needs to know', 'Fred is green.'), ('To determine if Charlie is kind, a person needs to know', 'Fred is blue.'), ('To determine if Charlie is kind, a person needs to know', 'Harry is round.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is nice and round and young then they are green.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is blue and kind then they are rough.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is green and kind and blue then they are nice.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is kind and cold and blue then they are big.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is rough and green then they are kind.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is kind and round then they are big.'), ('To determine if Charlie is kind, a person needs to know', 'If someone is blue and rough and kind then they are cold.')]}, {'guid': '8c6a277d-a1bf-46da-a984-89f8c2193b10', 'qa_pairs': [('Is it true or false that Charlie is not blue?', 'false')], 'facts': [('To determine if Charlie is not blue, a person needs to know', 'Charlie is kind.'), ('To determine if Charlie is not blue, a person needs to know', 'Charlie is green.'), ('To determine if Charlie is not blue, a person needs to know', 'Charlie is cold.'), ('To determine if Charlie is not blue, a person needs to know', 'Charlie is blue.'), ('To determine if Charlie is not blue, a person needs to know', 'Fred is red.'), ('To determine if Charlie is not blue, a person needs to know', 'Fred is kind.'), ('To determine if Charlie is not blue, a person needs to know', 'Fred is green.'), ('To determine if Charlie is not blue, a person needs to know', 'Fred is blue.'), ('To determine if Charlie is not blue, a person needs to know', 'Harry is round.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is nice and round and young then they are green.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is blue and kind then they are rough.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is green and kind and blue then they are nice.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is kind and cold and blue then they are big.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is rough and green then they are kind.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is kind and round then they are big.'), ('To determine if Charlie is not blue, a person needs to know', 'If someone is blue and rough and kind then they are cold.')]}

{'guid': 'd3747655-a840-434c-a1e2-af840c2d7636',
 'qa_pairs': [('Is it true or false that Bob is blue?', 'true')],
 'facts': [('To determine if Bob is blue, a person needs to know',
   'Bob is green.'),
  ('To determine if Bob is blue, a person needs to know', 'Bob is blue.'),
  ('To determine if Bob is blue, a person needs to know', 'Eric is round.'),
  ('To determine if Bob is blue, a person needs to know', 'Eric is rough.'),
  ('To determine if Bob is blue, a person needs to know', 'Eric is green.'),
  ('To determine if Bob is blue, a person needs to know', 'Gary is rough.'),
  ('To determine if Bob is blue, a person needs to know', 'Gary is red.'),
  ('To determine if Bob is blue, a person needs to know', 'Gary is nice.'),
  ('To determine if Bob is blue, a person needs to know', 'Gary is blue.'),
  ('To determine if Bob is blue, a person needs to know',
   'If someone is round and red then they are young.'),
  ('To determine if Bob is blue, a person needs to know',
   'If someone 

In [20]:
bos = "|starttext|"
eos = "|endtext|"
gen = "<gen>"

questions = [f"{data['qa_pairs'][0][0]} {gen} {data['qa_pairs'][0][1]}{eos}" for data in batch]
questions

['Is it true or false that Bob is blue? <gen> true|endtext|',
 'Is it true or false that Eric is not round? <gen> false|endtext|',
 'Is it true or false that Charlie is kind? <gen> true|endtext|',
 'Is it true or false that Charlie is not blue? <gen> false|endtext|']

In [12]:
facts_batch = [[fact[1] for fact in data['facts']] for data in batch]
facts_batch

[['Bob is green.',
  'Bob is blue.',
  'Eric is round.',
  'Eric is rough.',
  'Eric is green.',
  'Gary is rough.',
  'Gary is red.',
  'Gary is nice.',
  'Gary is blue.',
  'If someone is round and red then they are young.',
  'If someone is green and cold and red then they are blue.',
  'If someone is nice and blue then they are round.',
  'If someone is red and young and blue then they are cold.',
  'If someone is big and round and young then they are nice.',
  'If someone is green and blue and red then they are big.'],
 ['Bob is green.',
  'Bob is blue.',
  'Eric is round.',
  'Eric is rough.',
  'Eric is green.',
  'Gary is rough.',
  'Gary is red.',
  'Gary is nice.',
  'Gary is blue.',
  'If someone is round and red then they are young.',
  'If someone is green and cold and red then they are blue.',
  'If someone is nice and blue then they are round.',
  'If someone is red and young and blue then they are cold.',
  'If someone is big and round and young then they are nice.',
  

In [21]:
inputs = []
for facts, question in zip(facts_batch, questions):
    fact_prefix = f"{bos}"
    for fact in facts:
        fact_prefix += f"{fact} "
    inputs.append(f"{fact_prefix} {question}")
inputs

['|starttext|Bob is green. Bob is blue. Eric is round. Eric is rough. Eric is green. Gary is rough. Gary is red. Gary is nice. Gary is blue. If someone is round and red then they are young. If someone is green and cold and red then they are blue. If someone is nice and blue then they are round. If someone is red and young and blue then they are cold. If someone is big and round and young then they are nice. If someone is green and blue and red then they are big.  Is it true or false that Bob is blue? <gen> true|endtext|',
 '|starttext|Bob is green. Bob is blue. Eric is round. Eric is rough. Eric is green. Gary is rough. Gary is red. Gary is nice. Gary is blue. If someone is round and red then they are young. If someone is green and cold and red then they are blue. If someone is nice and blue then they are round. If someone is red and young and blue then they are cold. If someone is big and round and young then they are nice. If someone is green and blue and red then they are big.  Is i